In [ ]:
#| hide
#| default_exp gpfa.gpfa

# Gaussian Processes Factor Analysis

Derivation of the equations to solve the Gaussian Processes Factor Analysis as described in: Yu, B.M., Cunningham, J.P., Santhanam, G., Ryu, S., Shenoy, K.V., Sahani, M., 2008. Gaussian-process factor analysis for low-dimensional single-trial analysis of neural population activity, in: Advances in Neural Information Processing Systems. Curran Associates, Inc.

## Notation

- $T$  Number of time steps
- $N$  Number of variables observed
- $K$  Number of dimensions of latent variable
- $x_{:,t}$ vector of all the $N$ variables at time $t$, $\in \mathbb{R}^N$
- $x_{n,:}$ vector of the $n$th variable at for time steps in $T$, $\in \mathbb{R}^T$
- $x_{n,t}$ $n$th variable at time $t$, $\in \mathbb{R}$
- $X_M = [x_{:,1}, ... x_{:, T}]$ Matrix with all the $N$ variables at all time steps $T$, $\in \mathbb{R}^{N \times T}$
- $X$ is a vector obtained by "flattening" $X_M$, by putting next to each other all variable at time $t$, $\in \mathbb{R}^{(N \cdot T)}$
- $t$ time step
- $z_{i, t}$ $i$th latent variable at time $t$, $\in \mathbb{R}$
- $Z = [z_1 , ... z_t]$ Vector with $z$ at all time steps in $T$, $\in \mathbb{R}^{K \times T}$



## Gaussian Processes Factor Analysis model

We model the variables in this way
 $$x_{:,t} = \Lambda z_{:,t} + \epsilon $$
where:

- $\Lambda$ is a Factor loading matrix that transforms $z_{:,t}$ into $x_{:,t}$, $\in \mathbb{R}^{N \times K}$
- $\epsilon$ Random noise. The random noise is independent between the different time steps, $\in \mathbb{R}^N$:
    - $p(\epsilon) = \mathcal{N}(0, \psi)$ distribution of noise
    - $\psi$, covariance matrix of noise, it is a diagional matrix, $\in \mathbb{R}^{N \times N}$

The model consider $\langle X \rangle = 0$ (if $X$ doesn't have a 0 mean it can be easily transformed by substracting the mean)

The latent variable $z$ is modelled over time using a Gaussian Process, one process for each dimension $k$
for simplicity we assumed that $z$ has only one dimension ($k = 1$)

$$p(Z) = \mathcal{GP}(0, k(t, t \prime))$$

## Derivation of $p(X)$

$p(x_{:,t}|z_{:,t}) = \mathcal{N}(\Lambda z_{:,t}, \psi)$ is easy to derive and then $p(x_{:,t})$ and $p(z_{:,t}|x_{:,t})$ can be obtained using the rules of Gaussian inference.

However, what is interesting is to have the analytical form of $p(X)$, which models both the relations between $z$ and $x$ and the $z$ and $t$. The likelihood of $p(X)$ can then be maximized to obtain the parameters of the latent transformation and the kernel hyperparameter.

$p(X)$ is a Guassian distribution with $T\cdot N$ dimensions.

$p(X) = \mathcal{N}(\langle X \rangle, \langle X X^T \rangle)$




### Diagonal of the covariance matrix

Let's start with the diagonal of the covariance matrix ($t = t \prime$)

$\langle x_{:,t}x_{:,t}^T \rangle = \langle (\Lambda z_{:,t} + \epsilon_{t})(\Lambda z_{:,t} + \epsilon_{t})^T \rangle$

by multipling the two vectors together we obtain

$\langle x_{:,t}x_{:,t}^T \rangle = \langle \Lambda z_{:,t} z_{:,t}^T \Lambda^T + \Lambda z_{:,t} \epsilon_{t}^T + \epsilon_t \Lambda^T z_{:,t}^T  + \epsilon_t \epsilon_{t}^T \rangle$

The using the linearity of the [expectation](https://www.statlect.com/fundamentals-of-probability/expected-value-properties) we can:

1) transform the expecations of a sum into a sum of expecations
2) move the $\Lambda$ out of the expecation, as it doesn't depend on $z$
3) $\langle z_{:,t} \epsilon_t \rangle = \langle z_{:,t} \rangle \langle \epsilon_t \rangle$ because $z_{:,t}$ and $\epsilon_t$ are independent random variables

$\langle x_{:,t}x_{:,t}^T \rangle = \Lambda \langle z_{:,t} z_{:,t}^T\rangle \Lambda^T + \Lambda \langle z_{:,t} \rangle \langle \epsilon_{t}^T \rangle + \langle \epsilon_{t} \rangle \Lambda^T \langle z_{:,t}^T  \rangle + \langle \epsilon_t \epsilon_{t}^T \rangle$

Then considering that $\langle z_{:,t} \rangle = 0$ and that $\langle \epsilon_t \rangle = 0$
the expression can be simplified as:

$\langle x_{:,t}x_{:,t}^T \rangle = \Lambda \langle z_{:,t} z_{:,t}^T\rangle \Lambda^T + \langle \epsilon_t \epsilon_{t}^T \rangle$

Then substituting:

1) $\langle z_{:,t} z_{:,t}^T\rangle = k(t, t)$ as that is the covariance matrix of the Gaussian process
2) $\langle \epsilon_t \epsilon_t^T \rangle= \psi$

$\langle x_{:,t}x_{:,t}^T \rangle = \Lambda k(t,t)  \Lambda^T + \psi$

### Off-diagonal

similar to the steps of above

$\langle x_{:,t}x_{:,t \prime}^T \rangle = \langle (\Lambda z_{:,t} + \epsilon_{t})(\Lambda z_{:,t \prime} + \epsilon_{t \prime})^T \rangle$

by multipling the two vectors together we obtain

$\langle x_{:,t}x_{:,t \prime}^T \rangle = \langle \Lambda z_{:,t} z_{:,t \prime}^T \Lambda^T + \Lambda z_{:,t} \epsilon_{t \prime}^T + \epsilon_t \Lambda^T z_{:,t \prime}^T  + \epsilon_t \epsilon_{t \prime}^T \rangle$

Then using the linearity of the [expectation](https://www.statlect.com/fundamentals-of-probability/expected-value-properties) we can:

1) transform the expecations of a sum into a sum of expecations
2) move the $\Lambda$ out of the expecatios, as it doesn't depend on t 
3) $\langle z_{:,t} \epsilon_t \rangle = \langle z_{:,t} \rangle \langle \epsilon_t \rangle$ because $z_{:,t}$ and $\epsilon_t$ are independent random variables

$\langle x_{:,t}x_{:,t}^T \rangle = \Lambda \langle z_{:,t} z_{:,t}^T\rangle \Lambda^T + \Lambda \langle z_{:,t} \rangle \langle \epsilon_{t}^T \rangle + \langle \epsilon_{t} \rangle \Lambda^T \langle z_{:,t}^T  \rangle + \langle \epsilon_t \epsilon_{t}^T \rangle$

Then considering that $\langle z_{:,t} \rangle = 0$ and that $\langle \epsilon_t \rangle = 0$
the expression can be simplified as:

$\langle x_{:,t}x_{:,t \prime}^T \rangle = \Lambda \langle z_{:,t} z_{:,t \prime}^T\rangle \Lambda^T + \langle \epsilon_t \epsilon_{t \prime}^T \rangle$

Then substituting:
1) $\langle z_{:,t} z_{:,t \prime}^T\rangle = k(t,t \prime)$ as that is the covariance matrix of the Gaussian process
2) $\langle \epsilon_t \epsilon_{t \prime}^T \rangle= 0$ as $\epsilon_t$ and $\epsilon_{t \prime}$ are independent and $\langle \epsilon_t \rangle = 0$

$\langle x_{:,t}x_{:,t \prime}^T \rangle = \Lambda k(t,t \prime) \Lambda^T$


### Result

The equation for the diagonal and off-diagonal element can be summarized as:

$$\langle x_{:,t}x_{:,t \prime}^T \rangle = \Lambda k(t,t \prime) \Lambda^T + \delta(t - t \prime)\psi$$

where $\delta(x) = \begin{cases}
                1 & if\ x=0 \\
                0    & if\ x \ne 0 \\
            \end{cases}$ 

Therefore $p(X)$ can be modelled as:

$$p(X) = \mathcal{N}\left (0 , {\begin{array}{cccc}
    \Lambda k(t_1,t_1) \Lambda^T + \delta(1-1)\psi & \Lambda k(t_{1},t_{2}) \Lambda^T + \delta(1-2)\psi& \cdots & \Lambda k(t_1 ,t_t) \Lambda^T + \delta(1-t)\psi\\
    \Lambda k(t_{2},t_{1}) \Lambda^T+ \delta(2-1)\psi &  \Lambda k(t_{2},t_{2}) \Lambda^T + \delta(2-2)\psi & \cdots & \Lambda k(t_{2},t_{t}) \Lambda^T + \delta(2-t)\psi\\
    \vdots & \vdots & \ddots & \vdots\\
    \Lambda k(t_{t}, t_{1}) \Lambda^T+ \delta(t-1)\psi & \Lambda k(t_{t},t_{2}) \Lambda^T + \delta(t-2)\psi& \cdots & \Lambda k(t_{t},t_{t}) \Lambda^T + \delta(t-t)\psi\\
    \end{array} } \right )$$

and this is also Gaussian Process with a "special" kernel. Multiplying kernel with a constant ($\Lambda$) or adding a kernel ($\delta$) yields another valid kernel


If we define a new kernel as $$K(t,t \prime) = \Lambda k(t,t \prime) \Lambda^T + \delta(t - t \prime)\psi$$

Then

$$ p(X) = \mathcal{GP}(0, K(t, t\prime))$$

### Latent variable with more than one dimension

In order to have a latent variable with more than one-dimesions, we need to make small changes to the formula

The starting point is the covariance matrix of the latent at time $t$, which is:

$$\langle z_{:,t} z_{:,t}^T\rangle = \left(\begin{array}{ccc} \langle z_{1:,t} z_{1:,t}^T\rangle & \cdots & \langle z_{1:,t} z_{k:,t}^T\rangle\\ \vdots & \ddots & \vdots \\ \langle z_{k:,t} z_{1:,t}^T \rangle & \cdots & \langle z_{k:,t} z_{k:,t}^T\rangle\end{array}\right)$$ 

since each dimension in $z$ is indipendent:

$\langle z_{k,t} z_{k\prime,t} \rangle = 0$

each dimension in $z$ is modelled using a different kernel ($k_{z_k}(t,t\prime)$), hence:

$$\langle z_{:,t} z_{:,t}^T\rangle = \left(\begin{array}{ccc} k_{z_1}(t, t) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t, t)\end{array}\right)$$ 


so the GPFA Kernel is:   
$$K(t,t \prime) = \Lambda \left(\begin{array}{ccc} k_{z_1}(t, t \prime) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t, t \prime)\end{array}\right) \Lambda^T + \delta(t - t \prime)\psi$$

so `p(X)` is:

$$p(X) = \mathcal{N}\left (0 , {\begin{array}{ccc}
    \Lambda \left(\begin{array}{ccc} k_{z_1}(t_1, t_1) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t_1, t_1)\end{array}\right) \Lambda^T + \delta(t_1 - t_1)\psi & \cdots & \Lambda \left(\begin{array}{ccc} k_{z_1}(t_1, t_t) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t_1, t_t)\end{array}\right) \Lambda^T + \delta(t_1 - t_t)\psi\\
    \vdots & \ddots & \vdots\\
    \Lambda \left(\begin{array}{ccc} k_{z_1}(t_t, t_1) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t_t, t_1)\end{array}\right) \Lambda^T + \delta(t_t - t_1)\psi & \cdots & \Lambda \left(\begin{array}{ccc} k_{z_1}(t_t, t_t) & \cdots & 0 \\ \vdots & \ddots & \vdots \\ 0 & \cdots & k_{z_k}(t_t, t_t)\end{array}\right) \Lambda^T + \delta(t_t - t_t)\psi\\
    \end{array} } \right )$$

## Next steps

- The parameters of the final GP ($\Lambda, \psi$ and the kernel hyperparameters) can be fitted by maximizing the likelihood of $p(X)$ using gradient descent


# Implementation

In [ ]:
#| export
import torch
import gpytorch

from fastcore.foundation import patch
import pandas as pd

## Kernel

In [ ]:
#| exports
class GPFAKernel(gpytorch.kernels.Kernel):
    """
    Kernel to implement Gaussian Processes Factor Analysis
    """
    def __init__(self,
                 n_features: int, # number of variables at each time step
                 latent_kernel: gpytorch.kernels.Kernel, # func that returns any valid GPyTorch Kernel used to model the relationship over time of the latent
                 latent_dims:int = 1,  # Number of latent dims
                 Lambda: torch.tensor = None, #(n_features * latent_dims) initial value for factor loading matrix. If None init to one
                 psi: torch.tensor = None, #(n_features) initial value for random noise covariance. Note this is only the diagonal matrix
                 **kwargs):
        super(GPFAKernel, self).__init__(**kwargs)
        
        # Number of features in the X for each time step
        self.n_features = n_features

        self.latent_dims = latent_dims
        
        # see GPyTorch Kernels
        self.register_parameter(
            name = "Lambda",
            parameter = torch.nn.Parameter(torch.rand(self.n_features, self.latent_dims)))
        
        # each dim has it's own latent kernel
        self.latent_kernels = torch.nn.ModuleList([latent_kernel() for _ in range(self.latent_dims)])
        
        self.register_parameter(
            name = "raw_psi_diag",
            parameter = torch.nn.Parameter(torch.zeros(self.n_features))) 
        self.register_constraint("raw_psi_diag", gpytorch.constraints.Positive())
        if psi is not None: self.psi = psi
    
    # Convenient getter and setter for psi, since there is the Positive() constraint
    @property
    def psi(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_psi_diag_constraint.transform(self.raw_psi_diag)

    @psi.setter
    def psi(self, value):
        return self._set_psi(value)

    def _set_psi(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_psi_diag)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_psi_diag=self.raw_psi_diag_constraint.inverse_transform(value))
    

        
    def forward(self, t1, t2, diag = False, last_dim_is_batch=False, **params):

        # not implemented yet
        assert diag is False
        assert last_dim_is_batch is False

        # take the number of observations from the input
        n_obs = t1.shape[0]

        # compute the latent kernel
        kT = torch.stack([ kernel(t1, t2, diag, last_dim_is_batch, **params).evaluate() # this may make the whole thing slow as it breaks lazy evaluations
                         for kernel in self.latent_kernels], dim=2)
        return compute_gpfa_covariance(self.Lambda, kT, self.psi, self.n_features, n_obs)
    
    def num_outputs_per_input(self, x1,x2):
        return self.n_features

# this is a separate function, because torch script cannot take self as a parameter
@torch.jit.script
def compute_gpfa_covariance(Lambda, kT, psi, n_features, n_obs):
    # pre allocate covariance matrix
    X_cov = torch.empty(n_features * n_obs, n_features * n_obs, device=Lambda.device)
    for i in torch.arange(n_obs):
        for j in torch.arange(n_obs):
            # i:i+1 is required to keep the number of dimensions
            cov =  Lambda @ torch.diag(kT[i,j,:]) @ Lambda.T
            # only diagonals add the noise
            if i == j: cov += torch.diag(psi)
            # add a block of size n_features*n_features to the covariance matrix
            X_cov[i*n_features:(i*n_features + n_features),j*n_features:(j*n_features+n_features)] = cov
    return X_cov

In [ ]:
gpfa_k = GPFAKernel(n_features=2, latent_kernel=gpytorch.kernels.RBFKernel)

The parameters are correctly registered

In [ ]:
list(gpfa_k.named_parameters())

Check that the Kernel is running

In [ ]:
gpfa_k(torch.tensor((1, 2, 3))).evaluate()

## GPFA 

In [ ]:
#| export
class GPFAZeroMean(gpytorch.means.Mean):
    """
    Zero Mean function to be used in GPFA, as it takes into account the number of features
    """
    def __init__(self, n_features, device):
        super().__init__()
        self.n_features = n_features
        self.device = device
    def forward(self, input):
        shape = input.shape[0] * self.n_features
        return torch.zeros(shape, device=self.device)

to change the latent_kernel you should subcall `GPFA`
in this way the `get_info` function for the kernel can be changed to include the latent kernel details

In [ ]:
#| export
class GPFA(gpytorch.models.ExactGP):
    latent_kernel = gpytorch.kernels.RBFKernel
    def __init__(self, train_x, train_y, likelihood, n_features,latent_dims=1):
        super(GPFA, self).__init__(train_x, train_y, likelihood)
        self.likelihood = likelihood
        self.mean_module = GPFAZeroMean(n_features, train_x.device) # gets device from train_x
        self.covar_module = GPFAKernel(n_features, self.latent_kernel, latent_dims = latent_dims)

    def forward(self, x, **params):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x, **params)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

make some very simple test data, to check that the model is working and can learn the parameters

In [ ]:
T = torch.arange(1,5)

In [ ]:
X = torch.hstack([torch.arange(0,3) + 2* i for i in T]) 

In [ ]:
X

In [ ]:
X.shape

In [ ]:
T

In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPFA(T, X, likelihood, n_features = 3)

In [ ]:
model

Getting the prior from the GP

In [ ]:
model(T)

Fitting the parameters using gradient descend

In [ ]:
# this is for running the notebook in our testing framework
training_iter = 10

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
losses = []
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(T)
    # Calc loss and backprop gradients
    loss = -mll(output, X)
    losses.append(loss.item())
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f, Lambda: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.latent_kernels[0].lengthscale.item(),
        model.covar_module.Lambda.mean().item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

The model is training!

## Multi-dimensional latent variable

#### 2 dimensions

In [ ]:
# initialize likelihood and model
likelihood_m = gpytorch.likelihoods.GaussianLikelihood()
model_m = GPFA(T, X, likelihood, n_features = 3, latent_dims=2)

In [ ]:
model_m

check GP is running

In [ ]:
model_m.covar_module.latent_kernels

In [ ]:
model_m.covar_module.Lambda

In [ ]:
likelihood(model_m(T))

#### 5 dimensions

In [ ]:
# initialize likelihood and model
likelihood_m = gpytorch.likelihoods.GaussianLikelihood()
model_m = GPFA(T, X, likelihood, n_features = 3, latent_dims=5)

In [ ]:
model_m

check GP is running

In [ ]:
model_m.covar_module.latent_kernels

In [ ]:
model_m.covar_module.Lambda

In [ ]:
likelihood(model_m(T))

## Get info

this is to return the kernel info for printing, both during training and for the final results

The general api is:

`get_info(var_names=None) -> dict[str, pd.DataFrame]` where the string the title of a kernel parameter 

In [ ]:
#| exports
@patch
def get_info(self: GPFA,
             var_names = None # Optional variable names for better printing
            ) -> dict[str, pd.DataFrame]:
    "Model info for a GPFA with a RBFKernel"
    out = {}
    
    latent_names = [f"z{i}" for i in range(self.covar_module.latent_dims)]
    
    out["Lambda"] = pd.concat([
        None if var_names is None else pd.Series(var_names, name='variable'),
        pd.DataFrame(
            self.covar_module.Lambda.detach().cpu().numpy(),
            columns=latent_names)],
        axis=1)
    
    ls = [self.covar_module.latent_kernels[i].lengthscale.detach().item() for i in range(self.covar_module.latent_dims)]
    out["lengthscale"] = pd.DataFrame({
        'latent': latent_names,
        'lengthscale': ls
    })
    
    psi = self.covar_module.psi.detach().cpu().numpy()
    out["psi"] = pd.DataFrame({
        'variable': var_names,
        'psi': psi 
    })
    
    out["likelihood"] = pd.DataFrame({'noise': [self.likelihood.noise_covar.noise.detach().item()]})
    
    return out

In [ ]:
model_m.get_info()

In [ ]:
model_m.get_info(["a", "b", "c"])

## Export

In [ ]:
model_m.covar_module.latent_kernels

ModuleList(
  (0): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
  (1): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
  (2): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
  (3): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
  (4): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
)

In [ ]:
model_m.covar_module.Lambda

Parameter containing:
tensor([[0.1433, 0.6319, 0.2940, 0.8762, 0.6009],
        [0.7275, 0.7272, 0.2959, 0.3548, 0.8001],
        [0.9632, 0.0958, 0.9202, 0.0039, 0.7309]], requires_grad=True)

In [ ]:
likelihood(model_m(T))

MultivariateNormal(loc: torch.Size([12]))

## Get info

this is to return the kernel info for printing, both during training and for the final results

The general api is:

`get_info(var_names=None) -> dict[str, pd.DataFrame]` where the string the title of a kernel parameter 

In [ ]:
#| exports
@patch
def get_info(self: GPFA,
             var_names = None # Optional variable names for better printing
            ) -> dict[str, pd.DataFrame]:
    "Model info for a GPFA with a RBFKernel"
    out = {}
    
    latent_names = [f"z{i}" for i in range(self.covar_module.latent_dims)]
    
    out["Lambda"] = pd.concat([
        None if var_names is None else pd.Series(var_names, name='variable'),
        pd.DataFrame(
            self.covar_module.Lambda.detach().cpu().numpy(),
            columns=latent_names)],
        axis=1)
    
    ls = [self.covar_module.latent_kernels[i].lengthscale.detach().item() for i in range(self.covar_module.latent_dims)]
    out["lengthscale"] = pd.DataFrame({
        'latent': latent_names,
        'lengthscale': ls
    })
    
    psi = self.covar_module.psi.detach().cpu().numpy()
    out["psi"] = pd.DataFrame({
        'variable': var_names,
        'psi': psi 
    })
    
    out["likelihood"] = pd.DataFrame({'noise': [self.likelihood.noise_covar.noise.detach().item()]})
    
    return out

NameError: name 'patch' is not defined

In [ ]:
model_m.get_info()

{'Lambda':          z0        z1        z2        z3        z4
 0  0.143287  0.631874  0.294008  0.876215  0.600861
 1  0.727490  0.727183  0.295870  0.354760  0.800073
 2  0.963249  0.095809  0.920186  0.003902  0.730877,
 'lengthscale':   latent  lengthscale
 0     z0     0.693147
 1     z1     0.693147
 2     z2     0.693147
 3     z3     0.693147
 4     z4     0.693147,
 'psi':   variable       psi
 0     None  0.693147
 1     None  0.693147
 2     None  0.693147,
 'likelihood':       noise
 0  1.155176}

In [ ]:
model_m.get_info(["a", "b", "c"])

{'Lambda':    0        z0        z1        z2        z3        z4
 0  a  0.143287  0.631874  0.294008  0.876215  0.600861
 1  b  0.727490  0.727183  0.295870  0.354760  0.800073
 2  c  0.963249  0.095809  0.920186  0.003902  0.730877,
 'lengthscale':   latent  lengthscale
 0     z0     0.693147
 1     z1     0.693147
 2     z2     0.693147
 3     z3     0.693147
 4     z4     0.693147,
 'psi':   variable       psi
 0        a  0.693147
 1        b  0.693147
 2        c  0.693147,
 'likelihood':       noise
 0  1.155176}

## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()